Lower layer has QBCM+weight decay. Upper layer is a linear layer to learn reconstruction, and updated using gradient descent on L2 loss (do not back-propagate to lower layer). Trained on MNIST dataset, pre-centered at zero-mean, mini-batch size 100. Trained with a bottleneck dimension 40 (input dimension 784)

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
import matplotlib.pyplot as plt

% matplotlib inline

In [ ]:
# Build the network
n_input = 784
n_hidden1 = 40
n_output = 784

learning_rate = 0.0001
activation =tf.nn.relu
initializer = tf.contrib.layers.variance_scaling_initializer()

X=tf.placeholder(tf.float32, shape=(None, n_inputs))

weights1_init = initializer([n_inputs, n_hidden1])
weights2_init = initializer([n_hidden1, n_output])

weights1 = tf.Variable(weights1_init, dtype=tf.float32, name='weights1')
weights2 = tf.Variable(weights2_init, dtype=tf.float32, name='weights2')

hidden1 = activation(tf.matmul(X, weights1))
outputs = tf.matmul(hidden1, weights2)

optimizer = tf.train.AdamOptimizer(learning_rate)

# Train lower layer with QBCM rule
with tf.name_scope("phase1"):
    phase1_output = hidden1
    
    